NFL Project -> Extracted data from the espn website APIs. Use this github for api extraction:  Thank you by the way! Finding out what team_stats contribute most to winning. Definitely will answer more questions and give high-level interpretation of the data. Data is extracted for NFL season 2025 (regular season games only). Graphs and visualization will be implemented to clearly show the stats that contribute most to winning and maybe help NFL teams build better football teams for future seasons! Thanks for supporting and any further questions or suggestions would be greatly appreciated!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
from db import get_connection
from dotenv import load_dotenv

load_dotenv()

conn = get_connection()

In [ ]:
query = """
SELECT * FROM team_stats WHERE win is NOT NULL 
"""
df = pd.read_sql(query, conn)

In [ ]:
#sanity checks

df.shape

In [ ]:
df['win'].mean() #win category not exactly 50 percent which is strange 0.498
df['interceptions'].max() #4
df.groupby('team_id')['interceptions'].max() #interceptions thrown by each team
max_pass_yds=df.groupby('team_id')['tot_pass_yards'].max() #show all 32 teams highest passing yards
m=max_pass_yds.max()
print(m)
df[df['tot_pass_yards'] == m][['team_id', 'tot_pass_yards']] #find the team_id with the most passing yards (Rams)
